### Imports and loading 

In [1]:
import asyncio
import dspy
from prompt_optimization.seed_prompts import ATOMIZER_PROMPT, PLANNER_PROMPT, AGGREGATOR_PROMPT, ATOMIZER_DEMOS, PLANNER_DEMOS
from dspy import GEPA

from prompt_optimization import (
    get_default_config,
    LMConfig,
    patch_romaconfig,
    load_aimo_datasets,
    load_simpleqa_verified_dataset,
    load_frames_dataset,
    load_seal0_dataset,
    ComponentJudge,
    MetricWithFeedback,
    create_optimizer,
)
from prompt_optimization.seed_prompts import (
    ATOMIZER_PROMPT,
    ATOMIZER_DEMOS,
    PLANNER_PROMPT,
    PLANNER_DEMOS,
    AGGREGATOR_PROMPT,
)
from prompt_optimization.grader_prompt import GRADER_PROMPT
from roma_dspy.config import load_config
from roma_dspy.core.engine.solve import RecursiveSolver
from roma_dspy.core.modules.recursive_solver import RecursiveSolverModule
from roma_dspy.utils import AsyncParallelExecutor

dspy.settings.provide_traceback = True  # optional but mirrors the old notebook
opt_cfg = load_config(profile="test")

2025-10-17 13:54:37.128 | DEBUG    | roma_dspy.config.manager:load_config:57 - Loading config: path=None, profile=test, overrides=None, env_prefix=ROMA_
2025-10-17 13:54:37.128 | DEBUG    | roma_dspy.config.manager:load_config:66 - Initialized empty base config (defaults applied in validation)
2025-10-17 13:54:37.136 | DEBUG    | roma_dspy.config.manager:_load_yaml:129 - Loaded and cached config from config/defaults/config.yaml
2025-10-17 13:54:37.137 | DEBUG    | roma_dspy.config.manager:load_config:81 - Merged default config from config/defaults/config.yaml
2025-10-17 13:54:37.146 | DEBUG    | roma_dspy.config.manager:_load_yaml:129 - Loaded and cached config from config/profiles/test.yaml
2025-10-17 13:54:37.160 | DEBUG    | roma_dspy.config.manager:load_config:87 - Applied profile: test
2025-10-17 13:54:37.162 | DEBUG    | roma_dspy.config.manager:load_config:103 - Resolved interpolations
2025-10-17 13:54:37.163 | DEBUG    | roma_dspy.tools.base.manager:_register_toolkit_class:119 

### Config LLMS

In [2]:
# Batch the knobs you used to tweak in the notebook.
opt_cfg.train_size = 32
opt_cfg.val_size = 8
opt_cfg.test_size = 8
opt_cfg.dataset_seed = 42
opt_cfg.max_metric_calls = 225
opt_cfg.max_depth = 1
opt_cfg.enable_logging = False

In [3]:
#Add few-shot examples + prompts
opt_cfg.agents.atomizer.signature_instructions = ATOMIZER_PROMPT
opt_cfg.agents.planner.signature_instructions = PLANNER_PROMPT
opt_cfg.agents.aggregator.signature_instructions = AGGREGATOR_PROMPT

### Init solvers and what not

In [4]:
solver = RecursiveSolver(
    config=opt_cfg,
    max_depth=opt_cfg.max_depth,
    enable_logging=opt_cfg.enable_logging,
    enable_checkpoints=False,
)
solver_module = RecursiveSolverModule(solver=solver)

2025-10-17 13:54:42.425 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature for atomizer
2025-10-17 13:54:42.427 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:91 - Created atomizer agent (task_type=default, signature=default)
2025-10-17 13:54:42.427 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered atomizer instance #1 (task_type=default)
2025-10-17 13:54:42.428 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature for planner
2025-10-17 13:54:42.429 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:91 - Created planner agent (task_type=default, signature=default)
2025-10-17 13:54:42.430 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered planner instance #2 (task_type=default)
2025-10-17 13:54:42.432 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature fo

In [7]:
judge_lm = LMConfig("openrouter/anthropic/claude-sonnet-4.5", temperature=0.75, max_tokens=128000, cache=True)

In [8]:
judge = ComponentJudge(prompt=GRADER_PROMPT, lm_config=judge_lm)  # keyword required after the refactor
metric = MetricWithFeedback(judge)

In [9]:

train_set, val_set, test_set = load_frames_dataset(
    train_size=opt_cfg.train_size,
    val_size=opt_cfg.val_size,
    test_size=opt_cfg.test_size,
    seed=opt_cfg.dataset_seed,
)

### Perform an eval on the val set

In [10]:
# executor = AsyncParallelExecutor(max_concurrency=4)

# results = await executor.execute_batch(solver_module, test_set)

In [11]:
# print(results)

### Prompt tuning stuff

In [ ]:
optimizer = GEPA(
    metric=metric,
    # auto="light",
    component_selector="round_robin",
    max_metric_calls=32,
    add_format_failure_as_feedback=True,
    num_threads=6,
    track_stats=True,
    log_dir="logs/frames_test",
    # use_wandb=True,
    # wandb_init_kwargs={"project": "aime_test"},
    reflection_minibatch_size=8,
    reflection_lm=dspy.LM(model="openrouter/anthropic/claude-sonnet-4.5", temperature=.75, max_tokens=128000)
)

In [ ]:
optimized_program = optimizer.compile(
    solver_module,
    trainset=train_set,
    valset=val_set,
)

In [ ]:
optimized_program.named_predictors()